<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/tweepyaug10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python get_retweet_user_info.py --df politifact_agg.csv --label real --bs 4 --max-iter 5

import argparse

import json
import ast
import tweepy
import pandas as pd
from tqdm.auto import tqdm

from datetime import datetime

consumer_key = "sGqhuuHrWcoPleaKhBgAv6Iff"
consumer_secret = "kvTUnIkhf4jWCUOR73xSisYY3ksiT8eMFN4FBJxjqoBuhMjHkK"
access_key = "3289043624-Rme14R7m9XK4jIJywousUezoOc6s8WI5BrZkZoxG"
access_secret = "PleIICcykMGeySUnUYKiarUljnTltnXftUZ5Yz5TrvcfR"

def main(api, args):

    df = pd.read_csv(args.df)
    assert args.label in ('fake', 'real'), "category should be one of `fake`, `real`"

    df = df[df['label'] == args.label]
    df['retweets'] = df.retweets.map(ast.literal_eval)

    retweets = set()
    for l in df.retweets:
        l = [int(x) for x in l[:args.max_retweets]]
        retweets.update(set(l))

    retweets = sorted(list(retweets))

    if args.t2u is None:
        t2u = {}
    else:
        with open(args.t2u) as f:
            t2u = json.load(f)

    if args.users_dict is None:
        users_dict = {}
    else:
        with open(args.users_dict) as f:
            users_dict = json.load(f)

    if len(t2u):
        retweets = [x for x in retweets if str(x) not in t2u]
    print(f"{len(retweets)} retweets to process...")
    user_keys = ['screen_name', 'location', 'description', 'followers_count', 'friends_count', 'verified']

    for i in range(len(retweets)//args.bs):
        batch = retweets[i*args.bs: (i+1)*args.bs]
        tweet_infos = api.lookup_statuses(batch, include_entities=False, trim_user=False)
        if not len(tweet_infos):
            continue
        for tweet in tqdm(tweet_infos):
            uid = tweet.user.id
            t2u[tweet.id] = uid

            if uid in users_dict:
                continue
            user_info = {k:getattr(tweet.user, k, None) for k in user_keys}
            friends = []
            for page in tweepy.Cursor(api.get_friend_ids, user_id=uid).pages():
                friends.extend(page)
                if args.max_friends and len(friends) >= args.max_friends:
                    break
            followers = []
            for page in tweepy.Cursor(api.get_follower_ids, user_id=uid).pages():
                followers.extend(page)
                if args.max_followers and len(friends) >= args.max_followers:
                    break
            user_info['friends'] = friends
            user_info['followers'] = followers
            users_dict[uid] = user_info
        if i == args.max_iter:
            break
        # time.sleep(1200)

    with open('users_info.json', 'w') as f:
        json.dump(users_dict, f)
    with open('t2u.json', 'w') as f:
        json.dump(t2u, f)


if __name__=="__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument('C:\Users\salon\Downloads\politifact_agg.csv', type=str, help="Path to dataframe with `retweets` column")
    parser.add_argument('--t2u', type=str, help="Path to json file containing tweet-to-user mapping")
    parser.add_argument('--users-dict', type=str, help="Path to user info dictionary")
    parser.add_argument('--max-iter', type=int, default=1, help='Number of tweet_id batches to process')
    parser.add_argument('--max-friends', type=int, default=5000, help='Maximal number of friends to retrieve per user')
    parser.add_argument('--max-followers', type=int, defaultc=5000, help='Maximal number of followers to retrieve per user')
    parser.add_argument('--max_retweets', type=int, default=100, help="Maximal number of retweets per article to process")
    parser.add_argument('--label', type=str, help="Article label: fake or real")
    parser.add_argument('--bs', type=int, default=1, help="Batch size to process tweets")

    args = parser.parse_args()
    
    auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)

    main(api, args)



SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (Temp/ipykernel_27424/4272834908.py, line 88)